In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict({
    'month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'],
    'eggs': [33, 22, 13, 28, 43, 55],
    'salt': [12, 32,  9,  3,  23,  0], 
    'spam': [17, 31, 72, 20, 52, 55]
})
df.set_index('month', inplace=True)
df

# 基础

### Indexing value

In [ ]:
# 使用中括号
df["eggs"]['May']

# 使用列名
df.eggs['May']

# 使用 loc
df.loc['May', 'eggs']
df.loc['May']['eggs']

# 使用 iloc
df.iloc[4, 0]

### Slicing DataFrames

In [ ]:
# eggs 列中的某些值
df['eggs'][1:3]

# eggs 列中的某个值
df['eggs'][1]

# 某些列中的所有行
df.loc[:, 'eggs':'salt']

# 某些行中所有列
df.loc['Jan':'Feb', :]

# 某些行中某些列
df.loc['Jan':'Mar', 'eggs':'salt']
df.loc['Jan':'Mar', ['eggs', 'salt']]  # 使用列表删选行

# 使用 iloc 选择
df.iloc[2:5, 1:]

# df['eggs']  -> pandas.core.series.Series
# df[['eggs]] -> pandas.core.frame.DataFrame
pass

### Filtering

In [ ]:
df.loc[:, df.all()]           # 所有非0值
df.loc[:, df.any()]           # 0值行
df.loc[:, df.isnull().any()]  # 任何 na 行
df.loc[:, df.notnull().any()] # 任何 na 行
df.dropna(how='any')
pass

### Manipulating

In [ ]:
df.eggs[df.salt > 55] += 5   # 加
df.floordiv(12)              # 类似 dataframe // other，支持fill_value
pass

### About DataFrame Index

In [ ]:
# 创建 Series
prices = [3, 2, 1.5, 4, 4.5]
shares = pd.Series(prices)

# 创建Series，从数组指定索引值
days = ['Mon', 'Tue', 'Wed', 'Thur', 'Fri']
shares = pd.Series(prices, index=days)

In [ ]:
# 可使用下标访问索引值，也可以切片
shares.index[0]

# 给索引起名
shares.name = "Share"

# 给索引赋值会出错

In [ ]:
# 多重索引访问索引名
# df.index.name  -> None
# df.index.names -> ['a', 'b']

# 值可以按索引排序
df.sort_index()

# 多重索引选择，使用元组
df.loc[('idx1','idx2'), 'col_name']

# 只选择单个索引，会返回该索引内部得值（包括子索引）
df.loc['idx1']

# 使用 slice 函数，对第一层不设条件，返回第二层 ab 结果
df.loc[slice(None), slice('a','b')]

# Pivot

- Pivot 根据列值 Reshape 数据，有重复值会出错
- PivotTable 根据列值 Reshape 数据，有重复值可指定 aggregate 函数(sum, count, etc.)